In [ ]:
############ YAHOO FINANCE API #########

In [23]:

def fetch_stock_data(symbol, interval='2m', range_span='1d'):
    """
    Fetches stock data for a given symbol from Yahoo Finance.

    :param symbol: The stock symbol (e.g., 'NAUKRI.NS')
    :param interval: The interval for the stock data (default is '2m')
    :param range_span: The date range for the stock data (default is '1d')
    :return: A dictionary with stock data.
    """
    YAHOO_FINANCE_URL = f"https://query1.finance.yahoo.com/v8/finance/chart/{symbol}"

    params = {
        "region": "US",
        "lang": "en-US",
        "includePrePost": "false",
        "interval": interval,
        "useYfid": "true",
        "range": range_span,
        "corsDomain": "finance.yahoo.com",
        ".tsrc": "finance"
    }

    response = requests.get(YAHOO_FINANCE_URL, params=params)
    status_code = response.status_code

    if status_code != 200:
        raise RuntimeError(f"An error occurred. HTTP Code: {status_code}.")
    else:
        return response.json()

# Example usage
stock_symbol = 'NAUKRI.NS'  # Example stock symbol

try:
    stock_data = fetch_stock_data(stock_symbol)
    # Process and print relevant information from stock_data
    print(stock_data)
except RuntimeError as e:
    print(f"Error fetching data for {stock_symbol}: {e}")



Error fetching data for NAUKRI.NS: An error occurred. HTTP Code: 403.


In [ ]:
############ ALPHA VANTAGE API #############

In [ ]:
from alpha_vantage.timeseries import TimeSeries
from datetime import datetime
from config import ALPHA_VANTAGE_API_KEY  

def get_stock_data(symbol):
    app = TimeSeries('ALPHA_VANTAGE_API_KEY')
    data, meta_data = app.get_daily_adjusted(symbol, outputsize='compact')

    today = datetime.now().strftime('%Y-%m-%d')  # Get today's date in YYYY-MM-DD format

    if today in data:
        return {
            'symbol': symbol,
            'data': data[today]
        }
    else:
        print(f"No data available for {symbol} on {today}")
        return None

# Retrieve and store stock data for AAPL, META, and TSLA in an array
stock_prices_today = []

stock_symbols = ['AAPL', 'META', 'TSLA']
for symbol in stock_symbols:
    stock_data = get_stock_data(symbol)
    if stock_data is not None:
        stock_prices_today.append(stock_data)

# Displaying the array of stock prices
print(stock_prices_today)


[{'symbol': 'AAPL', 'data': {'1. open': '191.49', '2. high': '192.93', '3. low': '190.825', '4. close': '191.31', '5. adjusted close': '191.31', '6. volume': '39269515', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}}, {'symbol': 'META', 'data': {'1. open': '339.21', '2. high': '342.92', '3. low': '338.58', '4. close': '341.49', '5. adjusted close': '341.49', '6. volume': '10691386', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}}, {'symbol': 'TSLA', 'data': {'1. open': '242.04', '2. high': '244.01', '3. low': '231.4', '4. close': '234.21', '5. adjusted close': '234.21', '6. volume': '117754574', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}}]


In [ ]:
############ ALPHA VANTAGE API with requests #############

In [4]:
import requests
from datetime import datetime, timedelta
from config import ALPHA_VANTAGE_API_KEY  # Importing API key from config
import pprint  # Import pprint for pretty-printing

def get_stock_data(symbol):
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': symbol,
        'apikey': ALPHA_VANTAGE_API_KEY,  # Using imported API key
        'outputsize': 'compact'
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Calculate yesterday's date in YYYY-MM-DD format
    yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

    if 'Time Series (Daily)' in data and yesterday in data['Time Series (Daily)']:
        # Pretty-print only yesterday's data
        pprint.pprint({yesterday: data['Time Series (Daily)'][yesterday]})

        return {
            'symbol': symbol,
            'data': data['Time Series (Daily)'][yesterday]
        }
    else:
        print(f"No data available for {symbol} on {yesterday}")
        return None

# Retrieve and store previous day's stock data for AAPL, META, and TSLA in an array
stock_prices_yesterday = []

stock_symbols = ['AAPL', 'META', 'TSLA']
for symbol in stock_symbols:
    stock_data = get_stock_data(symbol)
    if stock_data is not None:
        stock_prices_yesterday.append(stock_data)

# Displaying the array of stock prices for the previous day
print(stock_prices_yesterday)


{'2023-11-27': {'1. open': '189.92',
                '2. high': '190.67',
                '3. low': '188.9',
                '4. close': '189.79',
                '5. adjusted close': '189.79',
                '6. volume': '40552609',
                '7. dividend amount': '0.0000',
                '8. split coefficient': '1.0'}}
{'2023-11-27': {'1. open': '336.18',
                '2. high': '339.9',
                '3. low': '334.2',
                '4. close': '334.7',
                '5. adjusted close': '334.7',
                '6. volume': '15684454',
                '7. dividend amount': '0.0000',
                '8. split coefficient': '1.0'}}
{'2023-11-27': {'1. open': '236.89',
                '2. high': '238.33',
                '3. low': '232.1',
                '4. close': '236.08',
                '5. adjusted close': '236.08',
                '6. volume': '112031763',
                '7. dividend amount': '0.0000',
                '8. split coefficient': '1.0'}}
[{'symbol

In [6]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from config import ALPHA_VANTAGE_API_KEY  # Importing API key from config

def get_stock_data(symbol):
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': symbol,
        'apikey': ALPHA_VANTAGE_API_KEY,  # Using imported API key
        'outputsize': 'compact'
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Calculate yesterday's date in YYYY-MM-DD format
    yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

    if 'Time Series (Daily)' in data and yesterday in data['Time Series (Daily)']:
        daily_data = data['Time Series (Daily)'][yesterday]
        # Flatten the data and add the symbol
        flattened_data = {f'{key.split(". ")[1]}': value for key, value in daily_data.items()}
        flattened_data['symbol'] = symbol
        flattened_data['date'] = yesterday  # Adding date for reference
        return flattened_data
    else:
        print(f"No data available for {symbol} on {yesterday}")
        return None

# Retrieve and store previous day's stock data in a DataFrame
stock_data_list = []

stock_symbols = ['AAPL', 'META', 'TSLA']
for symbol in stock_symbols:
    stock_data = get_stock_data(symbol)
    if stock_data is not None:
        stock_data_list.append(stock_data)

# Creating a DataFrame
df = pd.DataFrame(stock_data_list)

# Setting the index of the DataFrame to the symbol
df.set_index('symbol', inplace=True)

# Displaying the DataFrame
print(df)



          open    high    low   close adjusted close     volume  \
symbol                                                            
AAPL    189.92  190.67  188.9  189.79         189.79   40552609   
META    336.18   339.9  334.2   334.7          334.7   15684454   
TSLA    236.89  238.33  232.1  236.08         236.08  112031763   

       dividend amount split coefficient        date  
symbol                                                
AAPL            0.0000               1.0  2023-11-27  
META            0.0000               1.0  2023-11-27  
TSLA            0.0000               1.0  2023-11-27  


In [3]:
import requests
from datetime import datetime, timedelta
from config import ALPHA_VANTAGE_API_KEY
import pprint

def get_last_two_weeks_stock_data(symbol):
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': symbol,
        'apikey': ALPHA_VANTAGE_API_KEY,
        'outputsize': 'full'  # Change to 'full' to get more data
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Calculate the date two weeks ago
    two_weeks_ago = (datetime.now() - timedelta(weeks=2)).strftime('%Y-%m-%d')

    if 'Time Series (Daily)' in data:
        time_series = data['Time Series (Daily)']
        last_two_weeks_data = {date: details for date, details in time_series.items() if date >= two_weeks_ago}
        pprint.pprint(last_two_weeks_data)  # Pretty-print the two weeks' data

        return {
            'symbol': symbol,
            'data': last_two_weeks_data
        }
    else:
        print(f"No data available for {symbol}")
        return None

# Retrieve and store the last two weeks' stock data for AAPL, META, and TSLA
stock_prices_last_two_weeks = []

stock_symbols = ['AAPL', 'META', 'TSLA']
for symbol in stock_symbols:
    stock_data = get_last_two_weeks_stock_data(symbol)
    if stock_data is not None:
        stock_prices_last_two_weeks.append(stock_data)

# Display the array of stock prices for the last two weeks
pprint.pprint(stock_prices_last_two_weeks)


{'2023-11-20': {'1. open': '189.89',
                '2. high': '191.905',
                '3. low': '189.88',
                '4. close': '191.45',
                '5. adjusted close': '191.45',
                '6. volume': '46538614',
                '7. dividend amount': '0.0000',
                '8. split coefficient': '1.0'},
 '2023-11-21': {'1. open': '191.41',
                '2. high': '191.52',
                '3. low': '189.74',
                '4. close': '190.64',
                '5. adjusted close': '190.64',
                '6. volume': '38134485',
                '7. dividend amount': '0.0000',
                '8. split coefficient': '1.0'},
 '2023-11-22': {'1. open': '191.49',
                '2. high': '192.93',
                '3. low': '190.825',
                '4. close': '191.31',
                '5. adjusted close': '191.31',
                '6. volume': '39630011',
                '7. dividend amount': '0.0000',
                '8. split coefficient': '1.0'},
 '